In [1]:

import psycopg2
from psycopg2.extras import RealDictCursor
from datetime import datetime, date
import random


In [2]:
DB_CONFIG = {
    'host': 'localhost',
    'database': 'toolbox_db',
    'user': 'toolbox_user',
    'password': 'claude-key',
    'port': '5432'
}


In [3]:

# Connect to PostgreSQL database
conn = psycopg2.connect(**DB_CONFIG)
cursor = conn.cursor(cursor_factory=RealDictCursor)

# Enable autocommit for DDL operations
conn.autocommit = True
cursor.execute('''
    DROP TYPE IF EXISTS transaction_type_enum CASCADE;
    CREATE TYPE transaction_type_enum AS ENUM (
        'DEPOSIT', 'WITHDRAWAL', 'TRANSFER_IN', 'TRANSFER_OUT', 'FEE', 'INTEREST', 'CARD'
    );
''')

cursor.execute('''
    DROP TYPE IF EXISTS channel_enum CASCADE;
    CREATE TYPE channel_enum AS ENUM (
        'ATM', 'ONLINE', 'MOBILE', 'BRANCH', 'CARD', 'AUTO'
    );
''')

cursor.execute('''
    DROP TYPE IF EXISTS status_enum CASCADE;
    CREATE TYPE status_enum AS ENUM (
        'COMPLETED', 'FAILED', 'PENDING', 'CANCELLED'
    );
''')

# Drop table if exists
cursor.execute('DROP TABLE IF EXISTS transactions CASCADE;')

# Create the transactions table
cursor.execute('''
    CREATE TABLE transactions (
        transaction_id BIGSERIAL PRIMARY KEY,
        account_number VARCHAR(50) NOT NULL,
        transaction_date DATE NOT NULL,
        transaction_time TIME NOT NULL,
        transaction_type transaction_type_enum NOT NULL,
        amount NUMERIC(12, 2) NOT NULL,
        balance_after NUMERIC(12, 2),
        description TEXT,
        reference_number VARCHAR(50) UNIQUE NOT NULL,
        counterparty_account VARCHAR(50),
        counterparty_name VARCHAR(255),
        channel channel_enum,
        location VARCHAR(255),
        status status_enum NOT NULL DEFAULT 'COMPLETED',
        failure_reason VARCHAR(100),
        created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
    );
''')
#conn.autocommit = False

# Sample data for a single client
account_num = "ACC-123456789"
sample_transactions = [
    # Successful transactions
    (account_num, '2024-06-01', '09:15:00', 'DEPOSIT', 2500.00, 2500.00, 'Initial deposit', 'DEP-001', None, None, 'BRANCH', 'Main Branch', 'COMPLETED', None),
    (account_num, '2024-06-02', '14:30:00', 'WITHDRAWAL', -50.00, 2450.00, 'ATM withdrawal', 'WTH-001', None, None, 'ATM', 'Downtown ATM', 'COMPLETED', None),
    (account_num, '2024-06-03', '11:45:00', 'TRANSFER_IN', 300.00, 2750.00, 'Transfer from savings', 'TRF-001', 'SAV-123456789', 'Own Account', 'ONLINE', None, 'COMPLETED', None),
    # Failed transaction - insufficient funds
    (account_num, '2024-06-04', '18:22:00', 'WITHDRAWAL', -3000.00, None, 'ATM withdrawal attempt', 'WTH-002', None, None, 'ATM', 'Mall ATM', 'FAILED', 'INSUFFICIENT_FUNDS'),
    (account_num, '2024-06-05', '16:20:00', 'CARD', -25.50, 2724.50, 'Coffee shop purchase', 'CRD-001', 'MERCH-789', 'Starbucks #1234', 'CARD', 'New York, NY', 'COMPLETED', None),
    # Failed transaction - daily limit exceeded
    (account_num, '2024-06-06', '20:15:00', 'WITHDRAWAL', -500.00, None, 'ATM withdrawal attempt', 'WTH-003', None, None, 'ATM', 'Airport ATM', 'FAILED', 'DAILY_LIMIT_EXCEEDED'),
    (account_num, '2024-06-07', '10:00:00', 'TRANSFER_OUT', -1000.00, 1724.50, 'Rent payment', 'TRF-002', 'EXT-987654321', 'ABC Property Management', 'ONLINE', None, 'COMPLETED', None),
    # Failed transaction - network timeout
    (account_num, '2024-06-08', '13:45:00', 'TRANSFER_OUT', -200.00, None, 'Utility bill payment', 'TRF-003', 'EXT-456789123', 'Electric Company', 'MOBILE', None, 'FAILED', 'NETWORK_TIMEOUT'),
    # Failed transaction - invalid account
    (account_num, '2024-06-09', '11:30:00', 'TRANSFER_OUT', -150.00, None, 'Transfer to friend', 'TRF-004', 'EXT-999999999', 'John Smith', 'ONLINE', None, 'FAILED', 'INVALID_RECIPIENT_ACCOUNT'),
    (account_num, '2024-06-10', '08:30:00', 'FEE', -5.00, 1719.50, 'Monthly maintenance fee', 'FEE-001', None, None, 'AUTO', None, 'COMPLETED', None),
    # Failed transaction - card declined
    (account_num, '2024-06-12', '19:45:00', 'CARD', -89.99, None, 'Online purchase attempt', 'CRD-002', 'MERCH-456', 'Amazon.com', 'CARD', 'Online', 'FAILED', 'CARD_DECLINED_FRAUD_PROTECTION'),
    # Failed transaction - system maintenance
    (account_num, '2024-06-13', '02:15:00', 'TRANSFER_OUT', -75.00, None, 'Bill payment attempt', 'TRF-005', 'EXT-111222333', 'Phone Company', 'AUTO', None, 'FAILED', 'SYSTEM_MAINTENANCE'),
    (account_num, '2024-06-15', '12:00:00', 'INTEREST', 2.85, 1722.35, 'Monthly interest credit', 'INT-001', None, None, 'AUTO', None, 'COMPLETED', None),
    # Cancelled transaction - user cancelled
    (account_num, '2024-06-16', '14:20:00', 'TRANSFER_OUT', -300.00, None, 'Transfer cancelled by user', 'TRF-006', 'EXT-555666777', 'Investment Account', 'ONLINE', None, 'CANCELLED', 'USER_CANCELLED'),
]

# Insert sample data
cursor.executemany('''
    INSERT INTO transactions 
    (account_number, transaction_date, transaction_time, transaction_type, amount, balance_after, 
        description, reference_number, counterparty_account, counterparty_name, channel, location, status, failure_reason)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
''', sample_transactions)

# Commit changes
#conn.commit()
